<a href="https://colab.research.google.com/github/Tanguyvans/HackAI/blob/main/HackIA_Explicabilit%C3%A9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installer le module d'explicabilité "tf-explain"

In [24]:
!pip install grad-cam

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 21.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  U

# Importer les librairies

In [32]:
import torch
import cv2
from torchvision import transforms
from torchvision import models
from pytorch_grad_cam import GradCAM
from PIL import Image
import numpy as np
import collections

In [2]:
from matplotlib import pyplot as plt
import numpy as np
import tensorflow as tf
import tf_explain
from tf_explain.core.grad_cam import GradCAM
from tf_explain.core.smoothgrad import SmoothGrad
from keras.models import load_model
import cv2

# Télécharger la vidéo à expliquer

In [34]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [35]:
videos_to_test = "fire1.mp4"
cam_port = '/content/drive/MyDrive/Workshop/videos_tests/'+videos_to_test
cam = cv2.VideoCapture(cam_port)

# Télécharger le modèle de classification



In [36]:
filename = '/content/drive/MyDrive/Workshop/2. Image classification/model-Resnet-Epoch10.pth'
fire_model = torch.load(filename)

In [37]:
fire_model.eval()
target_layers = [fire_model.layer4[-1]]#ResNet : layer4[], MobileNet : features[]
gradcam = GradCAM(model=fire_model, target_layers=target_layers)

# Expliquer le modèle avec XAI "GradCAM" sur vidéo

In [38]:
classes = ["Fire", "No fire", "Start fire"]

In [42]:
transform = transforms.Compose(
    [
        transforms.ToPILImage(),
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]
)

In [40]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [46]:
while cam.isOpened():
  success, img = cam.read()
  if not success:
      break
  data = transform(img).to(device)
  data.unsqueeze_(0)
  with torch.inference_mode():
      classe = fire_model(data).cpu().detach().numpy()

  # si classe = fire ou smoke alors run le modèle d'explicabilité 'GradCam'
  if np.argmax(classe) == 0 or np.argmax(classe) == 2:
    # Appliquez GradCAM
    grayscale_cam = gradcam(input_tensor=data)

    # Assuming grayscale_cam is the CAM mask and img is the original PIL image
    rgb_img = np.array(img) / 255.0  # Normalize RGB image to [0, 1]
    # Resize the CAM mask to match the image dimensions
    grayscale_cam_resized = cv2.resize(grayscale_cam, (rgb_img.shape[1], rgb_img.shape[0]))

    cv2.putText(
        img,
        classes[np.argmax(classe)],
        (10, 50),
        cv2.FONT_HERSHEY_SIMPLEX,
        2,
        (0, 255, 0),
        thickness=3,
    )
  cv2.imwrite("XAI on Fire Classification", cv2.resize(img, (640, 480)))

  if cv2.waitKey(1) & 0xFF == ord("q"):
      break
cam.release()

error: OpenCV(4.8.0) /io/opencv/modules/imgcodecs/src/loadsave.cpp:696: error: (-2:Unspecified error) could not find a writer for the specified extension in function 'imwrite_'
